In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

import os
import glob

pd.options.display.float_format = '{:.4f}'.format

# Seed
np.random.seed(480)


# Load Clean Data
file_list = glob.glob("data/clean_*.csv") 

df = pd.concat([pd.read_csv(file) for file in file_list], # list comprehension, supposedly faster than for loop
               ignore_index=True
               )

df.isna().sum()

# drop rows with missing values in MOST_SEVERE_INJURY
df = df.dropna(subset=['MOST_SEVERE_INJURY'])


## Variables
factor_cols = [
    'CRASH_DATE_EST_I',
    #'POSTED_SPEED_LIMIT', 
    'TRAFFIC_CONTROL_DEVICE', 
    'DEVICE_CONDITION', 
    'WEATHER_CONDITION', 
    'LIGHTING_CONDITION', 
    'FIRST_CRASH_TYPE', ##### attention to this one! what to do?? keep in x?
    'TRAFFICWAY_TYPE', 
    'ALIGNMENT', 
    'ROADWAY_SURFACE_COND', 
    'ROAD_DEFECT', 
    'REPORT_TYPE', 
    'CRASH_TYPE', 
    'INTERSECTION_RELATED_I',
    'NOT_RIGHT_OF_WAY_I',
    'HIT_AND_RUN_I',
    'DAMAGE', ### attention to this one! potential response var??
    'PRIM_CONTRIBUTORY_CAUSE', 
    'SEC_CONTRIBUTORY_CAUSE',  
    'DOORING_I',
    'WORK_ZONE_I',
    'WORK_ZONE_TYPE', ### > 99% are N/A
    'WORKERS_PRESENT_I',
    'MOST_SEVERE_INJURY', 
    'CRASH_HOUR', 
    'CRASH_DAY_OF_WEEK', 
    'CRASH_MONTH',
    'Crash_Year', # Recently Incuded #
    'Police_district', # Recently Inc, using instead of 'BEAT_OF_OCCURRENCE'
    'LANE_CNT', # Recently Incuded #
]

# Convert to categorical
for col in factor_cols:
    df[col] = df[col].astype('category')


# 5 variables not useful for predictive analysis
not_usefull = ['CRASH_RECORD_ID', 
               'BEAT_OF_OCCURRENCE',
               'STREET_NO', 
               'STREET_DIRECTION', 
               'STREET_NAME',
               'LATITUDE', 
               'LONGITUDE',
               ]


# 2 Date variables
date_vars = ['CRASH_DATE', 
             'DATE_POLICE_NOTIFIED',
             ] # using Report_vs_Police_Notified: hours between crash and notification


# 8 potential response variables
responses = ['CRASH_TYPE', #1
             'MOST_SEVERE_INJURY', #2
             'INJURIES_TOTAL', #3
             'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
             #'DAMAGE',
             ]


In [ ]:
df

In [ ]:
# RIVERNORTH
df_state = df[df['STREET_NAME'] == "STATE ST"]
df_state = df_state[df_state['STREET_DIRECTION'] != "S"]
df_state = df_state[df_state['STREET_NO'] > 310]
df_state = df_state[df_state['STREET_NO'] < 640]
df_state = df_state[df_state['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_KINZIE = df[df['STREET_NAME'] == "KINZIE ST"]
df_KINZIE = df_KINZIE[df_KINZIE['STREET_NO'] < 15]
df_KINZIE = df_KINZIE[df_KINZIE['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_HUBBARD = df[df['STREET_NAME'] == "HUBBARD ST"]
df_HUBBARD = df_HUBBARD[df_HUBBARD['STREET_NO'] < 15]
df_HUBBARD = df_HUBBARD[df_HUBBARD['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_ILLINOIS = df[df['STREET_NAME'] == "ILLINOIS ST"]
df_ILLINOIS = df_ILLINOIS[df_ILLINOIS['STREET_NO'] < 15]
df_ILLINOIS = df_ILLINOIS[df_ILLINOIS['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_GRAND = df[df['STREET_NAME'] == "GRAND AVE"]
df_GRAND = df_GRAND[df_GRAND['STREET_NO'] < 15]
df_GRAND = df_GRAND[df_GRAND['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_OHIO = df[df['STREET_NAME'] == "OHIO ST"]
df_OHIO = df_OHIO[df_OHIO['STREET_NO'] < 15]
df_OHIO = df_OHIO[df_OHIO['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_ONTARIO = df[df['STREET_NAME'] == "ONTARIO ST"]
df_ONTARIO = df_ONTARIO[df_ONTARIO['STREET_NO'] < 15]
df_ONTARIO = df_ONTARIO[df_ONTARIO['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_rivernorth = pd.concat([df_state, df_KINZIE, df_HUBBARD, df_ILLINOIS, df_GRAND, df_OHIO, df_ONTARIO])

#df_rivernorth

In [ ]:
# RAVENSWOOD
df_ASHLAND = df[df['STREET_NAME'] == "ASHLAND AVE"]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_DIRECTION'] == "N"]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_NO'] > 4240]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_NO'] < 4620]
df_ASHLAND = df_ASHLAND[df_ASHLAND['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_WILSON = df[df['STREET_NAME'] == "WILSON AVE"]
df_WILSON = df_WILSON[df_WILSON['STREET_DIRECTION'] == "W"]
df_WILSON = df_WILSON[df_WILSON['STREET_NO'] > 1530]
df_WILSON = df_WILSON[df_WILSON['STREET_NO'] < 1620]
df_WILSON = df_WILSON[df_WILSON['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Sunnyside Ave
df_SUNNYSIDE = df[df['STREET_NAME'] == "SUNNYSIDE AVE"]
df_SUNNYSIDE = df_SUNNYSIDE[df_SUNNYSIDE['STREET_DIRECTION'] == "W"]
df_SUNNYSIDE = df_SUNNYSIDE[df_SUNNYSIDE['STREET_NO'] > 1530]
df_SUNNYSIDE = df_SUNNYSIDE[df_SUNNYSIDE['STREET_NO'] < 1620]
df_SUNNYSIDE = df_SUNNYSIDE[df_SUNNYSIDE['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Montrose
df_MONTROSE = df[df['STREET_NAME'] == "MONTROSE AVE"]
df_MONTROSE = df_MONTROSE[df_MONTROSE['STREET_DIRECTION'] == "W"]
df_MONTROSE = df_MONTROSE[df_MONTROSE['STREET_NO'] > 1530]
df_MONTROSE = df_MONTROSE[df_MONTROSE['STREET_NO'] < 1620]
df_MONTROSE = df_MONTROSE[df_MONTROSE['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Cullom
df_CULLOM = df[df['STREET_NAME'] == "CULLOM AVE"]
df_CULLOM = df_CULLOM[df_CULLOM['STREET_DIRECTION'] == "W"]
df_CULLOM = df_CULLOM[df_CULLOM['STREET_NO'] > 1530]
df_CULLOM = df_CULLOM[df_CULLOM['STREET_NO'] < 1620]
df_CULLOM = df_CULLOM[df_CULLOM['INTERSECTION_RELATED_I'].apply(str) == "Y"]



# Combine new dataframes
df_ravenswood = pd.concat([df_ASHLAND, df_WILSON, df_SUNNYSIDE, df_MONTROSE, df_CULLOM])
df_ravenswood

In [ ]:
# AUSTIN
df_GRAND = df[df['STREET_NAME'] == "GRAND AVE"]
df_GRAND = df_GRAND[df_GRAND['STREET_DIRECTION'] == "W"]
df_GRAND = df_GRAND[df_GRAND['STREET_NO'] < 6010]
df_GRAND = df_GRAND[df_GRAND['STREET_NO'] > 5950]
df_GRAND = df_GRAND[df_GRAND['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_AUSTIN = df[df['STREET_NAME'] == "AUSTIN AVE"]
df_AUSTIN = df_AUSTIN[df_AUSTIN['STREET_DIRECTION'] == "N"]
df_AUSTIN = df_AUSTIN[df_AUSTIN['STREET_NO'] < 2220]
df_AUSTIN = df_AUSTIN[df_AUSTIN['STREET_NO'] > 2170]
df_AUSTIN = df_AUSTIN[df_AUSTIN['INTERSECTION_RELATED_I'].apply(str) == "Y"]


# Combine new dataframes
df_austin = pd.concat([df_GRAND, df_AUSTIN])

In [9]:
# Humboldt Park
df_DIVISION = df[df['STREET_NAME'] == "DIVISION ST"]
df_DIVISION = df_DIVISION[df_DIVISION['STREET_DIRECTION'] == "W"]
df_DIVISION = df_DIVISION[df_DIVISION['STREET_NO'] < 2615]
df_DIVISION = df_DIVISION[df_DIVISION['STREET_NO'] > 2550]
df_DIVISION = df_DIVISION[df_DIVISION['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_ROCKWELL = df[df['STREET_NAME'] == "ROCKWELL ST"]
df_ROCKWELL = df_ROCKWELL[df_ROCKWELL['STREET_DIRECTION'] == "N"]
df_ROCKWELL = df_ROCKWELL[df_ROCKWELL['STREET_NO'] < 1220]
df_ROCKWELL = df_ROCKWELL[df_ROCKWELL['STREET_NO'] > 1160]
df_ROCKWELL = df_ROCKWELL[df_ROCKWELL['INTERSECTION_RELATED_I'].apply(str) == "Y"]


# Combine new dataframes
df_humboldt = pd.concat([df_DIVISION, df_ROCKWELL])
df_humboldt


,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,Report_vs_Police_Notified,Crash_Year,Police_district
253879,4775b7f9036d124c462dc26a7e864639506840c05bd88d...,N,2023-03-19 16:20:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,...,0.0000,5.0000,16,1,3,41.9029,-87.6921,0.1667,2023,District 12
275334,4dfb6a2e2be037879e99cf9921a3818ebc408a51a4e6d2...,N,2020-11-24 14:30:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,0.0000,2.0000,14,3,11,41.9031,-87.6921,1.1667,2020,District 14
312752,594dc496fac7600d8c713f409fa99cc0371e217948c0bb...,N,2018-12-25 08:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,NOT DIVIDED,...,0.0000,4.0000,8,3,12,41.9031,-87.6921,0.0833,2018,District 14
368403,a050aff9f56ed698275e0787cdd587f7d55fa19ea64afd...,N,2021-10-17 13:30:00,30,TRAFFIC SIGNAL,UNKNOWN,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,...,0.0000,2.0000,13,1,10,41.9031,-87.6921,48.0000,2021,District 14
437639,b55f50217ede3e9be8e319bba80f8363971646cb63f5cb...,N,2022-05-07 20:10:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0000,4.0000,20,7,5,41.9029,-87.6921,0.2333,2022,District 12
444992,b799df9d8a7361979523583f50b4d92696a956c3c40e6d...,N,2016-03-04 14:02:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,1.0000,1.0000,14,6,3,41.9029,-87.6921,0.8000,2016,District 14
556841,017d7547647cf989624ff6084b457cccfa6835a83dfe22...,N,2022-08-05 23:46:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0000,2.0000,23,6,8,41.9031,-87.6917,46.1333,2022,District 14
711152,3041c017760127b37e89ba5a8e533b53460e433da07a54...,Y,2018-04-03 19:57:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,0.0000,2.0000,19,3,4,41.9031,-87.6921,0.0500,2018,District 14
725336,d6a58eb6563e62bb66553b0f7fb7c439e7d88557bf9411...,N,2020-07-03 13:50:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0000,1.0000,13,6,7,41.9029,-87.6918,0.1667,2020,District 12
766364,e30d5fedc72596a9d25b44d7fc18ff47f30edc4447ec8c...,N,2020-02-06 15:30:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,PEDALCYCLIST,FOUR WAY,...,0.0000,1.0000,15,5,2,41.9031,-87.6921,0.0000,2020,District 14


In [13]:
# UKRANIAN VILLAGE
df_CHICAGO = df[df['STREET_NAME'] == "CHICAGO AVE"]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_DIRECTION'] == "W"]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_NO'] < 2320]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_NO'] > 2170]
df_CHICAGO = df_CHICAGO[df_CHICAGO['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_OAKLEY = df[df['STREET_NAME'] == "OAKLEY BLVD"]
df_OAKLEY = df_OAKLEY[df_OAKLEY['STREET_DIRECTION'] == "N"]
df_OAKLEY = df_OAKLEY[df_OAKLEY['STREET_NO'] < 820]
df_OAKLEY = df_OAKLEY[df_OAKLEY['STREET_NO'] > 750]
df_OAKLEY = df_OAKLEY[df_OAKLEY['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_LEAVITT = df[df['STREET_NAME'] == "LEAVITT ST"]
df_LEAVITT = df_LEAVITT[df_LEAVITT['STREET_DIRECTION'] == "N"]
df_LEAVITT = df_LEAVITT[df_LEAVITT['STREET_NO'] < 820]
df_LEAVITT = df_LEAVITT[df_LEAVITT['STREET_NO'] > 750]
df_LEAVITT = df_LEAVITT[df_LEAVITT['INTERSECTION_RELATED_I'].apply(str) == "Y"]


# Combine new dataframes
df_ukranian = pd.concat([df_CHICAGO, df_OAKLEY, df_LEAVITT])

In [14]:
# UKRANIAN VILLAGE 2
df_CHICAGO = df[df['STREET_NAME'] == "CHICAGO AVE"]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_DIRECTION'] == "W"]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_NO'] < 1720]
df_CHICAGO = df_CHICAGO[df_CHICAGO['STREET_NO'] > 1650]
df_CHICAGO = df_CHICAGO[df_CHICAGO['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_PAULINA = df[df['STREET_NAME'] == "PAULINA ST"]
df_PAULINA = df_PAULINA[df_PAULINA['STREET_DIRECTION'] == "N"]
df_PAULINA = df_PAULINA[df_PAULINA['STREET_NO'] < 820]
df_PAULINA = df_PAULINA[df_PAULINA['STREET_NO'] > 750]
df_PAULINA = df_PAULINA[df_PAULINA['INTERSECTION_RELATED_I'].apply(str) == "Y"]


# Combine new dataframes
df_ukranian2 = pd.concat([df_CHICAGO, df_PAULINA])


In [ ]:
# CHICAGO LAWN
df_63RD = df[df['STREET_NAME'] == "63RD ST"]
df_63RD = df_63RD[df_63RD['STREET_DIRECTION'] == "W"]
df_63RD = df_63RD[df_63RD['STREET_NO'] < 2420]
df_63RD = df_63RD[df_63RD['STREET_NO'] > 2350]
df_63RD = df_63RD[df_63RD['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_WESTERN = df[df['STREET_NAME'] == "WESTERN AVE"]
df_WESTERN = df_WESTERN[df_WESTERN['STREET_DIRECTION'] == "S"]
df_WESTERN = df_WESTERN[df_WESTERN['STREET_NO'] < 6320]
df_WESTERN = df_WESTERN[df_WESTERN['STREET_NO'] > 6250]
df_WESTERN = df_WESTERN[df_WESTERN['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_chicago_lawn = pd.concat([df_63RD, df_WESTERN])


df_63RD = df[df['STREET_NAME'] == "63RD ST"]
df_63RD = df_63RD[df_63RD['STREET_DIRECTION'] == "W"]
df_63RD = df_63RD[df_63RD['STREET_NO'] < 3220]
df_63RD = df_63RD[df_63RD['STREET_NO'] > 3150]
df_63RD = df_63RD[df_63RD['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_KEDZIE = df[df['STREET_NAME'] == "KEDZIE AVE"]
df_KEDZIE = df_KEDZIE[df_KEDZIE['STREET_DIRECTION'] == "S"]
df_KEDZIE = df_KEDZIE[df_KEDZIE['STREET_NO'] < 6320]
df_KEDZIE = df_KEDZIE[df_KEDZIE['STREET_NO'] > 6250]
df_KEDZIE = df_KEDZIE[df_KEDZIE['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_chicago_lawn = pd.concat([df_chicago_lawn, df_63RD, df_KEDZIE])


df_63RD = df[df['STREET_NAME'] == "63RD ST"]
df_63RD = df_63RD[df_63RD['STREET_DIRECTION'] == "W"]
df_63RD = df_63RD[df_63RD['STREET_NO'] < 2820]
df_63RD = df_63RD[df_63RD['STREET_NO'] > 2750]
df_63RD = df_63RD[df_63RD['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_CALIFORNIA = df[df['STREET_NAME'] == "CALIFORNIA AVE"]
df_CALIFORNIA = df_CALIFORNIA[df_CALIFORNIA['STREET_DIRECTION'] == "S"]
df_CALIFORNIA = df_CALIFORNIA[df_CALIFORNIA['STREET_NO'] < 6320]
df_CALIFORNIA = df_CALIFORNIA[df_CALIFORNIA['STREET_NO'] > 6250]
df_CALIFORNIA = df_CALIFORNIA[df_CALIFORNIA['INTERSECTION_RELATED_I'].apply(str) == "Y"]


# Combine new dataframes
df_chicago_lawn = pd.concat([df_chicago_lawn, df_63RD, df_CALIFORNIA])


In [ ]:
# ENGLEWOOD
df_71ST = df[df['STREET_NAME'] == "71ST ST"]
df_71ST = df_71ST[df_71ST['STREET_DIRECTION'] == "W"]
df_71ST = df_71ST[df_71ST['STREET_NO'] < 1610]
df_71ST = df_71ST[df_71ST['STREET_NO'] > 1570]
df_71ST = df_71ST[df_71ST['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_ASHLAND = df[df['STREET_NAME'] == "ASHLAND AVE"]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_DIRECTION'] == "S"]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_NO'] < 7120]
df_ASHLAND = df_ASHLAND[df_ASHLAND['STREET_NO'] > 7040]
df_ASHLAND = df_ASHLAND[df_ASHLAND['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_englewood = pd.concat([df_71ST, df_ASHLAND])

df_71ST = df[df['STREET_NAME'] == "71ST ST"]
df_71ST = df_71ST[df_71ST['STREET_DIRECTION'] == "W"]
df_71ST = df_71ST[df_71ST['STREET_NO'] < 2010]
df_71ST = df_71ST[df_71ST['STREET_NO'] > 1970]
df_71ST = df_71ST[df_71ST['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_DAMEN = df[df['STREET_NAME'] == "DAMEN AVE"]
df_DAMEN = df_DAMEN[df_DAMEN['STREET_DIRECTION'] == "S"]
df_DAMEN = df_DAMEN[df_DAMEN['STREET_NO'] < 7120]
df_DAMEN = df_DAMEN[df_DAMEN['STREET_NO'] > 7040]
df_DAMEN = df_DAMEN[df_DAMEN['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_englewood = pd.concat([df_englewood, df_71ST, df_DAMEN])

df_71ST = df[df['STREET_NAME'] == "71ST ST"]
df_71ST = df_71ST[df_71ST['STREET_DIRECTION'] == "W"]
df_71ST = df_71ST[df_71ST['STREET_NO'] < 810]
df_71ST = df_71ST[df_71ST['STREET_NO'] > 770]
df_71ST = df_71ST[df_71ST['INTERSECTION_RELATED_I'].apply(str) == "Y"]

df_HALSTED = df[df['STREET_NAME'] == "HALSTED AVE"]
df_HALSTED = df_HALSTED[df_HALSTED['STREET_DIRECTION'] == "S"]
df_HALSTED = df_HALSTED[df_HALSTED['STREET_NO'] < 7120]
df_HALSTED = df_HALSTED[df_HALSTED['STREET_NO'] > 7040]
df_HALSTED = df_HALSTED[df_HALSTED['INTERSECTION_RELATED_I'].apply(str) == "Y"]

# Combine new dataframes
df_englewood = pd.concat([df_englewood, df_71ST, df_HALSTED])
df_englewood


In [17]:
# CONCAT ALL OF THEM
df_final = pd.concat([df_ravenswood, df_humboldt, df_ukranian, df_ukranian2, df_chicago_lawn, df_englewood, df_austin])
df_final.shape

# SAVE AS csv
df_final.to_csv("data/lttc.csv", index=False)
df_rivernorth.to_csv("data/rivernorth.csv", index=False)